In [81]:
import torch
import torch.nn as nn
from torch.nn import functional as F

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = list(sorted(set(text)))


stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
stoi["["] = len(chars)+1
itos[len(chars)+1] = "["


encode = lambda s: [stoi[ch] for ch in s]
decode = lambda c: "".join([itos[i] for i in c])



split_ratio = 0.9
batch_size = 8
emb_size = 384
seq_len = 128
trans_layers = 6
num_head = 6
max_iters = 1000
eval_interval = 100
eval_iters = 200
vocab_size = len(chars)
learning_rate = 3e-4
device = "cuda"
mask_ratio = 0.2

train_data = text[:int(split_ratio*len(text))]
test_data = text[int(split_ratio*len(text)):]



def get_batch(batch_size, seq_len, split):
    X = []
    Y = []
    init_pos = torch.randint(0, len(train_data)-seq_len, size=[batch_size, ]) if split== "train" else torch.randint(0, len(test_data)-seq_len, size=[batch_size, ])
    for pos in init_pos:
        X.append(encode(train_data[pos:pos+seq_len])) if split == "train" else X.append(encode(test_data[pos:pos+seq_len])) 
        Y.append(encode(train_data[pos:pos+seq_len])) if split == "train" else Y.append(encode(test_data[pos:pos+seq_len])) 

    mask_num = int(seq_len * mask_ratio)
    
    return_pos = torch.zeros(batch_size, seq_len, dtype=bool)
    mask_positions = torch.randint(0, seq_len, size = [batch_size, mask_num])
    print(mask_positions)
    for i in range(batch_size):
        for pos in mask_positions[i]:
            X[i][pos] = len(chars)+1
            return_pos[i][pos] = True

    X = torch.tensor(X).to(device)
    Y = torch.tensor(Y).to(device)
    
    return X, return_pos, Y

X, mask_positions, Y = get_batch(batch_size, seq_len, "train")
input = torch.randn((batch_size, seq_len, 256))
print(input.shape)
print(mask_positions.shape)
print(input[mask_positions].shape)

tensor([[ 34,  34,  99,  45,  64,  71,   3, 102, 123, 112,  19,  92,  13, 108,
         121, 103, 126, 104,  27,  76,  22,  32, 124,   3, 127],
        [116, 111,  92, 123,  63,  64,  38,  89,  42,  93, 118,  44,  93,  17,
          64,  30,  46, 100,  46,  51,  89,  99, 126,  85,  80],
        [ 98,  20,  94, 105,   1,  17,   8,  73,  41,  60,  76,  24, 111,  44,
          54, 110,  23,  63, 110,  59,  17,  90,  76, 113,  95],
        [ 45,  54,  50, 105,   8,  93, 102,  73,  44,  45,  34,  59,  23,  97,
          67,  76,  54,  76,  69,  96,  24,  32,  98,  97,  65],
        [ 99,  37,  60, 120,  26,  54,  67,  11,  78,  39,  71,  93,   9, 114,
          58,  90,  35,  91,  47,  84,  34,  99,  68,  72,  75],
        [ 14,   8, 111,  26, 100,  35,  91,  99, 127,  60,  43,  22,  91,  20,
          64,   2,  41,  79,  18,  53,  95, 123, 124, 111, 112],
        [ 58,  39, 109,  98,  26, 125,  58,  14, 100, 103,  45,  81, 118, 116,
          79, 100,  51, 127,  90,  29, 123,  59, 118,  62